In [1]:
import sys
sys.path.append('/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages')

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# PART 1: Scrape data w BeautifulSoup

base_url='https://www.eld.gov.sg/'
url=requests.get(base_url+'elections_past_parliamentary.html')
soup = BeautifulSoup(url.content, 'html.parser')
year=''
links = []
for i in soup.find_all('a'):
    if i.text[:2]=='19' or i.text[:2]=='20':
        links.append((base_url+i.get('href'),i.text.replace('*','')))

def populate_raw_data(soup):
    raw = pd.DataFrame(columns=['year','constituency','electors','candidates','party','votes'])
#     each constituency under one h3 header
    for i in soup.find_all('h3'):
        const = i.text
#         electors count in bold, voting results in table below
        for sibling in i.next_siblings:
            if sibling.name == 'b':
                electors = sibling.text
            if sibling.name == 'div':
                nominees_all = sibling.find_all('tr')
                for nom in nominees_all:
                    data = nom.find_all('td')
                    if len(data)==3:
                        candidates = data[0]
                        party = data[1]
                        votes = data[2]
                        row = {'year':year,
                               'constituency':const,
                               'electors':electors,
                               'party':party,
                               'candidates':candidates,
                               'votes':votes}
                        raw = raw.append(row,ignore_index=True)
            if sibling.name=='h3':
                break   
    return raw

raw_all = pd.DataFrame(columns=['year','constituency','electors','party','candidates','votes'])

for link in links:
    url=requests.get(link[0])
    soup = BeautifulSoup(url.content, 'html.parser')
    year = link[1]
    raw_all = pd.concat([raw_all,populate_raw_data(soup)],sort=False)

raw_all

,year,constituency,electors,party,candidates,votes
0,2020,Aljunied,"Number of Electors: 150,821","[\n, [PAP], \n, [], \n]","[\r\n\t\t\t\t\t\t\t\t\t\t\t\t\tCHAN HUI YUH, [...","[57,330, [], (40.05%)]"
1,2020,Aljunied,"Number of Electors: 150,821","[\n, [WP], \n, [], \n]",[\r\n\t\t\t\t\t\t\t\t\t\t\t\t\tGERALD GIAM YEA...,"[85,815, [], (59.95%)]"
2,2020,Ang Mo Kio,"Number of Electors: 185,261","[\n, [PAP], \n, [], \n]","[\r\n\t\t\t\t\t\t\t\t\t\t\t\t\tDARRYL DAVID, [...","[124,597, [], (71.91%)]"
3,2020,Ang Mo Kio,"Number of Electors: 185,261","[\n, [RP], \n, [], \n]",[\r\n\t\t\t\t\t\t\t\t\t\t\t\t\tKENNETH ANDREW ...,"[48,677, [], (28.09%)]"
4,2020,Bishan-Toa Payoh,"Number of Electors: 101,220","[\n, [PAP], \n, [], \n]","[\r\n\t\t\t\t\t\t\t\t\t\t\t\t\tCHEE HONG TAT, ...","[62,983, [], (67.23%)]"
...,...,...,...,...,...,...
74,1955,Ulu Bedok,"Number of Electors: 16,903","[[PP], []]","[A. Hamid Rahmat, []]","[652, [], (6.96%)]"
75,1955,Ulu Bedok,"Number of Electors: 16,903","[[DP], []]","[Toh Seng Sit, []]","[2,999, [], (32.00%)]"
76,1955,Whampoa,"Number of Electors: 12,345","[[SLF], []]","[Chew Swee Kee, []]","[2,961, [], (45.88%)]"
77,1955,Whampoa,"Number of Electors: 12,345","[[DP], []]","[Lee Kok Liang, []]","[927, [], (14.37%)]"


In [3]:
# PART 2: Clean raw data columns

def get_vote_cnt(n):
    stats = n.text.split('(')
    votes = stats[0].replace(',','')
    try:
        return int(votes)
    except:
        return 0

def get_candidate_list(n):
    return [name.replace('<td>','') for name in str(n).split('<br/>')[:-1]]

clean_all = raw_all.copy()
clean_all['electors'] = clean_all['electors'].apply(lambda x:int(x.split()[-1].replace(',','')))
clean_all['votes']= clean_all['votes'].apply(get_vote_cnt)
clean_all['candidates']= clean_all['candidates'].apply(get_candidate_list)
clean_all['party'] = clean_all['party'].apply(lambda x:x.find('a').text)

candidate_data = clean_all.set_index(["year","constituency","electors","party","votes"]).explode('candidates')
candidate_data['candidates'] = candidate_data['candidates'].apply(lambda x:x.strip())
candidate_data['candidates'] = candidate_data['candidates'].apply(lambda x:x.upper())
candidate_data = candidate_data.reset_index()

In [ ]:
# PART 3: Format & Manipulate data

summary = clean_all.reset_index()[['year','constituency','party','electors','votes']].drop_duplicates()

# Votes cast, no. of electors per constituency

overall_votes = summary.groupby(['year','constituency','electors']).sum().reset_index()
overall_votes.rename(columns={"votes":"total_votes"},inplace=True)
overall_votes['perc_voted']=overall_votes['total_votes']/overall_votes['electors']

# Votes per constituency per party + indicate winner

party_votes = summary.groupby(['year','constituency','electors','party']).sum().reset_index()

winners = summary.groupby(['year','constituency','electors']).max()[['votes']]
winners['won']='Y'
winners.reset_index(inplace=True)

vote_results = party_votes.merge(winners,on=["year","constituency","electors","votes"],how="left")
vote_results['won'] = vote_results['won'].fillna('N')

# constituency type and size (GRC vs SMC)

ctype = clean_all.reset_index()
ctype = ctype[ctype['party']!='-'].groupby(['year','constituency','party']).nunique()[['candidates']]
grc = ctype[ctype['candidates']>1].reset_index()[['year','constituency','candidates']].drop_duplicates()

# Overall summary of election results

result_summary = overall_votes.merge(vote_results)
result_summary['perc_votes_won'] = result_summary['votes']/result_summary['total_votes']

result_summary = result_summary.merge(grc,how='left')
result_summary['candidates'] = result_summary['candidates'].apply(lambda x: x if x>1 else 1)
result_summary['type'] = result_summary['candidates'].apply(lambda x: 'GRC' if x>1 else 'SMC')

result_summary

,year,constituency,electors,total_votes,perc_voted,party,votes,won,perc_votes_won,candidates,type
0,1955,Bukit Panjang,8012,4289,0.535322,PP,3097,Y,0.722080,1.0,SMC
1,1955,Bukit Panjang,8012,4289,0.535322,SLF,1192,N,0.277920,1.0,SMC
2,1955,Bukit Timah,9173,6213,0.677314,DP,1308,N,0.210526,1.0,SMC
3,1955,Bukit Timah,9173,6213,0.677314,PAP,3259,Y,0.524545,1.0,SMC
4,1955,Bukit Timah,9173,6213,0.677314,PP,722,N,0.116208,1.0,SMC
...,...,...,...,...,...,...,...,...,...,...,...
1519,2020,West Coast,146089,138654,0.949106,PSP,66996,N,0.483188,5.0,GRC
1520,2020,Yio Chu Kang,25962,24294,0.935752,PAP,14775,Y,0.608175,1.0,SMC
1521,2020,Yio Chu Kang,25962,24294,0.935752,PSP,9519,N,0.391825,1.0,SMC
1522,2020,Yuhua,21351,20032,0.938223,PAP,14131,Y,0.705421,1.0,SMC


In [ ]:
# PART 4: Load into RDS. Generate insert row commands

# postgres command requires escape characters for candidate names with ' 
candidate_data['check']= candidate_data['candidates'].apply(lambda x: "E" if "'" in x else "")
candidate_data['candidate_name']= candidate_data['candidates'].apply(lambda x:x.replace("'",r"\'"))

load_party_data = 'insert into elections.party_data values '
for i in range(len(clean_all)):
    row = clean_all.iloc[i]
    load_party_data = load_party_data + """({0}, '{1}', {2}, '{3}', {4}), """.format(row['year'],row['constituency'],row['electors'],row['party'],row['votes'])

load_cand_data = 'insert into elections.candidate_data values '
for i in range(len(candidate_data)):
    row = candidate_data.iloc[i]
    load_cand_data = load_cand_data + """({0}, '{1}', '{2}', {3}'{4}'), """.format(row['year'],row['constituency'],row['party'],row['check'],row['candidate_name'])